### Load Data for Modelling

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import timedelta
import calendar
import math
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import xgboost as xgb
from joblib import dump, load
from sklearn import preprocessing
import category_encoders as ce

In [2]:
item_store_sales_df = pd.read_csv('/home/jupyter/script/train_sales4/train_sales.csv')
item_store_promo_df = pd.read_csv('/home/jupyter/script/train_promo1/final_promo.csv')
test_df = pd.read_csv('/home/jupyter/script/test.csv')
items_df = pd.read_csv('/home/jupyter/script/items.csv')
stores_df = pd.read_csv('/home/jupyter/script/stores.csv')

### Function

In [3]:
def get_data(data, dt_end, days, period, freq='D'):
  '''
  This function gives us the selected columns based on a range of dates passed.
  '''
  # for col in pd.date_range(dt_end - datetime.timedelta(days = days), periods = period, freq = freq):
  #     print(str(col)[0:10])
  # print("-------------------")
  return data[[str(col)[0:10] for col in pd.date_range(dt_end - datetime.timedelta(days = days), periods = period, freq = freq)]]

In [4]:
def average(data):
  '''
  Here we are calculating simple average
  '''
  return np.mean(data, axis = 1)

In [5]:
def feature_engg_sales(data, end_date, prefix):
  '''
  This function generates feature dictionary for train, cv, test
  Features generated are:
  moving average, weighted moving average, standard deviation observed, 
  moving average of DOW, weighted moving average of DOW, having total sales day,
  last sales day in n days, first sales day in n days
  '''
  # days_list = [3, 7, 16, 30]
  days_list = [3, 7, 14, 28]
  feature_dict = {'{}_average_{}_days'.format(prefix, days): average(get_data(data, end_date, days, days).values)  for days in days_list}
  # feature_dict.update({'{}_WMA_{}_days'.format(prefix, days): weighted_moving_average(get_data(data, end_date, days, days)) for days in days_list})
  # feature_dict.update({'{}_std_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).std(axis = 1).values for days in days_list})
  feature_dict.update({'{}_6avgdow_{}_days'.format(prefix, day) : get_data(data, end_date, 42 - day, 6, freq = '7D').mean(axis =1).values for day in range(7)})
  # feature_dict.update({'{}_6WMAdow_{}_days'.format(prefix, day) : weighted_moving_average(get_data(data, end_date, 42 - day, 6, freq = '7D')) for day in range(7)})
  # feature_dict.update({'{}_has_sale_day_{}'.format(prefix, days) : (get_data(data, end_date, days, days) > 0).sum(axis = 1).values for days in days_list})
  # feature_dict.update({'{}_last_has_sale_day_{}'.format(prefix, days) : days - ((get_data(data, end_date, days, days) > 0) * np.arange(days)).max(axis = 1).values for days in days_list})
  # feature_dict.update({'{}_first_has_sale_day_{}'.format(prefix, days) : ((get_data(data, end_date, days, days) > 0) * np.arange(days, 0, -1)).max(axis = 1).values for days in days_list})

  return feature_dict

In [6]:
def feature_engg_promo(data, end_date, prefix):
    '''
    This function uses promo information and categorical array to create features
    features created are---
    promo: total_promo, future promo information, promo days in 15 days, last promo in 15 days, first promo in 15 days
    categorical: class, item, store, family, city, state, clsuter, type 
    '''
    days_list = [3, 7, 14, 28]
    feature_dict = {}
    feature_dict = {'{}_totalpromo_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).sum(axis = 1).values for days in days_list}
    feature_dict.update({'{}_totalpromoafter_{}_days'.format(prefix, days) : get_data(data, end_date + timedelta(days = 16), 16, days).sum(axis = 1).values for days in [3, 7, 14]})
    feature_dict.update({'{}_promo_{}_day'.format(prefix, abs(day)): get_data(data, end_date, day, 1).values.ravel() for day in range(14)})
    feature_dict.update({'{}_promoafter_{}_day'.format(prefix, abs(day - 1)): get_data(data, end_date, day, 1).values.ravel() for day in range(-13, 1)})
    # feature_dict.update({'promo_day_in_15_days' : (get_data(data, end_date + timedelta(days=16), 15, 15) > 0).sum(axis = 1).values})
    # feature_dict.update({'last_promo_day_in_15_days' : 15 - ((get_data(data, end_date + timedelta(days=16), 15, 15) > 0) * np.arange(15)).max(axis = 1).values})
    # feature_dict.update({'firt_promo_day_in_15_days' : ((get_data(data, end_date + timedelta(days=16), 15, 15) > 0) * np.arange(15, 0, -1)).max(axis = 1).values})
    
    return feature_dict

### Preparing Train Data

In [7]:
#To create training points we will take multiple intervals and will concat all the information we got from these intervals....
x_lst, y_lst = [], []
num_of_intervals = 8
dates = [date(2017, 5, 31) + timedelta(days=7 * interval) for interval in range(num_of_intervals)]
for train_date in tqdm(dates):
  train_dict = feature_engg_sales(item_store_sales_df, train_date,'item_store')
  x_lst.append(pd.DataFrame(train_dict, index = [i for i in range(len(list(train_dict.values())[0]))]))
  y_lst.append(item_store_sales_df[[str(col)[0:10] for col in pd.date_range(train_date, periods = 16)]].values)

train_item_store_x = pd.concat(x_lst, axis=0)
train_y = np.concatenate(y_lst, axis=0)
del x_lst, y_lst
print(train_item_store_x.shape, train_y.shape)

100% 8/8 [00:02<00:00,  3.39it/s]

(1289952, 11) (1289952, 16)


In [8]:
x_lst = []
num_of_intervals = 8
dates = [date(2017, 5, 31) + timedelta(days=7 * interval) for interval in range(num_of_intervals)]
for train_date in tqdm(dates):
  train_dict = feature_engg_promo(item_store_promo_df, train_date,'item_store')
  x_lst.append(pd.DataFrame(train_dict, index = [i for i in range(len(list(train_dict.values())[0]))]))

train_item_store_x1 = pd.concat(x_lst, axis=0)
del x_lst
print(train_item_store_x1.shape)

100% 8/8 [00:02<00:00,  3.82it/s]

(1289952, 35)


In [9]:
train_dict = feature_engg_promo(item_store_promo_df, date(2017, 5, 31),'item_store')

In [10]:
# train_x = train_item_store_x
train_x = train_item_store_x.reset_index(drop = True).merge(train_item_store_x1.reset_index(drop = True), left_index=True, right_index=True)

In [11]:
[train_x[col].update((train_x[col] - train_x[col].min()) / (train_x[col].max() - train_x[col].min())) for col in train_x.columns]

/tmp/ipykernel_416/1478882014.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  [train_x[col].update((train_x[col] - train_x[col].min()) / (train_x[col].max() - train_x[col].min())) for col in train_x.columns]
/tmp/ipykernel_416/1478882014.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Preparing CV Data

In [12]:
#Generating sales features
cv_date = date(2017, 7, 26)
cv_dict = feature_engg_sales(item_store_sales_df, cv_date, 'item_store')
cv_item_store_x = pd.DataFrame(cv_dict, index = [i for i in range(len(list(cv_dict.values())[0]))])
cv_item_store_x.shape

(161244, 11)

In [13]:
#Generating promo and categorical features
cv_dict = feature_engg_promo(item_store_promo_df, cv_date, 'item_store')
cv_item_store_x1 = pd.DataFrame(cv_dict, index = [i for i in range(len(list(cv_dict.values())[0]))])
cv_item_store_x1.shape

(161244, 35)

In [14]:
#Merging all the data points
# cv_x = cv_item_store_x
cv_x = cv_item_store_x.reset_index(drop = True).merge(cv_item_store_x1.reset_index(drop = True), left_index=True, right_index=True)

In [15]:
[cv_x[col].update((cv_x[col] - cv_x[col].min()) / (cv_x[col].max() - cv_x[col].min())) for col in cv_x.columns]

/tmp/ipykernel_416/3150344123.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  [cv_x[col].update((cv_x[col] - cv_x[col].min()) / (cv_x[col].max() - cv_x[col].min())) for col in cv_x.columns]
/tmp/ipykernel_416/3150344123.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [16]:
#Generating y_i for cv
cv_y = item_store_sales_df[[str(col)[0:10] for col in pd.date_range(cv_date, periods = 16)]].values

### Preparing Test Data

In [17]:
#gathering sales featres
test_date = date(2017, 8, 16)
test_dict = feature_engg_sales(item_store_sales_df, test_date, 'item_store')
test_item_store_x = pd.DataFrame(test_dict, index = [i for i in range(len(list(test_dict.values())[0]))])
test_item_store_x.shape

(161244, 11)

In [18]:
test_dict = feature_engg_promo(item_store_promo_df, test_date, 'item_store')
test_item_store_x1 = pd.DataFrame(test_dict, index = [i for i in range(len(list(test_dict.values())[0]))])
test_item_store_x1.shape

(161244, 35)

In [19]:
# test_x = test_item_store_x
test_x = test_item_store_x.reset_index(drop = True).merge(test_item_store_x1.reset_index(drop = True), left_index=True, right_index=True)

In [20]:
[test_x[col].update((test_x[col] - test_x[col].min()) / (test_x[col].max() - test_x[col].min())) for col in test_x.columns]

/tmp/ipykernel_416/3580788485.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  [test_x[col].update((test_x[col] - test_x[col].min()) / (test_x[col].max() - test_x[col].min())) for col in test_x.columns]
/tmp/ipykernel_416/3580788485.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Modelling

In [21]:
test_pred = []
for i in range(train_y.shape[1]):
    print('step{}'.format(i+1))
    lr = LinearRegression()
    lr.fit(train_x, train_y[: , i])
    test_pred.append(lr.predict(test_x))

step1
step2
step3
step4
step5
step6
step7
step8
step9
step10
step11
step12
step13
step14
step15
step16


In [41]:
#Creating prediction df
y_test = np.array(test_pred).transpose()
pred_df = pd.DataFrame(y_test, columns=pd.date_range("2017-08-16", periods=16))

,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,0.151936,0.179521,0.221319,0.271277,0.242285,0.231164,0.185256,0.193216,0.194244,0.233738,0.278507,0.260639,0.237321,0.197339,0.217541,0.224860
1,0.281486,0.279720,0.451807,0.344925,0.172150,0.238819,0.252200,0.332907,0.318857,0.483594,0.383670,0.209381,0.272566,0.281445,0.371240,0.348728
2,0.597347,0.703794,0.859484,0.717960,0.403865,0.545584,0.615994,0.645001,0.737528,0.892967,0.763076,0.453521,0.582046,0.645662,0.685994,0.771246
3,1.042944,0.957914,1.183838,1.038444,0.756408,0.933835,0.931789,1.077311,0.975408,1.195049,1.059949,0.768977,0.945502,0.942996,1.100486,0.989906
4,1.679507,1.644417,1.868925,1.652066,1.205936,1.560285,1.577826,1.706483,1.656442,1.880723,1.680922,1.222320,1.574243,1.591190,1.737507,1.675362


In [23]:
item_store_sales_df['store_nbr'] = pd.to_numeric(item_store_sales_df['store_nbr'])
items_df['class'] = pd.to_numeric(items_df['class'])

In [24]:
#Melting down the predicted values based on dates
pred_df = item_store_sales_df[['item_nbr', 'store_nbr']].merge(pred_df, left_index=True, right_index=True)
pred_df = pred_df.melt(id_vars=['item_nbr', 'store_nbr'], var_name='date', value_name='unit_sales')
pred_df = pred_df.merge(items_df[['item_nbr', 'class']], how = 'left', on = 'item_nbr')
pred_df['unit_sales'] = pred_df['unit_sales'].apply(lambda x : np.expm1(x))

In [46]:
pred_df['date'] = pd.to_datetime(pred_df['date'])

In [47]:
#Reading test_file
test_df = pd.read_csv('/home/jupyter/script/test.csv')
test_df['date'] = pd.to_datetime(test_df['date'])
test_df.shape

(3370464, 5)

In [27]:
#Merging with the predicted values
test_df = test_df.merge(pred_df[['item_nbr', 'store_nbr', 'date', 'unit_sales']], on = ['date', 'store_nbr', 'item_nbr'], how = 'left')
test_df['unit_sales'] = test_df['unit_sales'].clip(lower = 0)
#Filling null values with 0
test_df = test_df.fillna(0)
#Making submission file
test_df[['id', 'unit_sales']].to_csv('/home/jupyter/submission/lr_submission.csv', index = False)